# Importing Libraries

In [1]:
import os
import shutil
import json
import pandas as pd
import numpy as np

# Question 1

### Files copied to zomato_raw_files
* mkdir -p /home/jaskeerat/zomato_raw_files
* cp /mnt/c/Users/jaske/Downloads/zomato_raw/* /home/jaskeerat/zomato_raw_files/

# Question 2

In [2]:
raw_files = "/home/jaskeerat/zomato_raw_files"
file_list = [files for files in os.listdir(raw_files) if files.split(".")[1] == "json"]
file_list

['file3.json', 'file1.json', 'file2.json', 'file4.json', 'file5.json']

In [3]:
raw_json_folder = "/home/jaskeerat/zomato_etl/source/json/"
os.makedirs(raw_json_folder, exist_ok=True)

for file in os.listdir(raw_files):
    if(file.split(".")[1] == "json"):
        source_file = os.path.join(raw_files, file)
        destination_file = os.path.join(raw_json_folder, file)
        shutil.copy(source_file, destination_file)
        print(source_file, destination_file)

/home/jaskeerat/zomato_raw_files/file3.json /home/jaskeerat/zomato_etl/source/json/file3.json
/home/jaskeerat/zomato_raw_files/file1.json /home/jaskeerat/zomato_etl/source/json/file1.json
/home/jaskeerat/zomato_raw_files/file2.json /home/jaskeerat/zomato_etl/source/json/file2.json
/home/jaskeerat/zomato_raw_files/file4.json /home/jaskeerat/zomato_etl/source/json/file4.json
/home/jaskeerat/zomato_raw_files/file5.json /home/jaskeerat/zomato_etl/source/json/file5.json


In [4]:
raw_csv_folder = "/home/jaskeerat/zomato_etl/source/csv/"
os.makedirs(raw_csv_folder, exist_ok=True)

file_mapping = {"file1.json": "zomato_20190609.csv", "file2.json": "zomato_20190610.csv", "file3.json": "zomato_20190611.csv", "file4.json": "zomato_20190612.csv", "file5.json": "zomato_20190613.csv"}

for json_file, csv_file in file_mapping.items():
    data_list = []
    with open(raw_json_folder + json_file, "r") as file:
        data = json.load(file)
        for d in data:
            if(("results_found" in d) and (d["results_found"] != 0)):
                data_list = data_list + d["restaurants"]

    df = pd.DataFrame()
    for data in data_list:
        restaurant_data = data["restaurant"]
        df = df._append({"Restaurant_ID": restaurant_data["R"]["res_id"], "Restaurant_Name": restaurant_data["name"], "Country_Code": restaurant_data["location"]["country_id"],
                        "City": restaurant_data["location"]["city"], "Address": restaurant_data["location"]["address"], "Locality": restaurant_data["location"]["locality"],
                        "Locality_Verbose": restaurant_data["location"]["locality_verbose"], "Longitude": restaurant_data["location"]["longitude"],
                        "Latitude": restaurant_data["location"]["latitude"], "Cuisines": restaurant_data["cuisines"], "Average_Cost_for_two": restaurant_data["average_cost_for_two"],
                        "Currency": restaurant_data["currency"], "Has_Table_booking": restaurant_data["has_table_booking"], "Has_Online_delivery": restaurant_data["has_online_delivery"],
                        "Is_delivering_now": restaurant_data["is_delivering_now"], "Switch_to_order_menu": restaurant_data["switch_to_order_menu"],
                        "Price_range": restaurant_data["price_range"], "Aggregate_rating": restaurant_data["user_rating"]["aggregate_rating"],
                        "Rating_text": restaurant_data["user_rating"]["rating_text"], "Votes": restaurant_data["user_rating"]["votes"]}, ignore_index = True)
    df.to_csv(raw_csv_folder + csv_file, index=False)

## Question 3

* hdfs dfs -mkdir -p /zomato_etl/zomato_ext/zomato
* hdfs dfs -put /home/jaskeerat/zomato_etl/source/csv/* /zomato_etl/zomato_ext/zomato

* hdfs dfs -mkdir -p /zomato_etl/zomato_ext/dim_country
* hdfs dfs -put /home/jaskeerat/zomato_raw_files/country_code.csv /zomato_etl/zomato_ext/dim_country

## Question 4